In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
# Upload JSON-file
with open("experiment_output.json", "r") as file:
    data = json.load(file)

# Make DataFrame
df = pd.DataFrame(data)

In [4]:
df.head(10)

,time_point,steps_count,heart_rate,skin_temp,event_flag
0,0,0,60.0,36.5,sleep
1,1,0,58.0,36.4,sleep
2,2,0,59.0,36.3,sleep
3,3,0,57.0,36.2,sleep
4,4,0,56.0,36.1,sleep
5,5,0,55.0,36.0,sleep
6,6,10,65.0,36.2,normal
7,7,500,120.0,37.0,workout
8,8,2000,100.0,37.2,normal
9,9,3000,90.0,37.3,normal


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time_point   24 non-null     int64  
 1   steps_count  24 non-null     int64  
 2   heart_rate   24 non-null     float64
 3   skin_temp    24 non-null     float64
 4   event_flag   24 non-null     object 
dtypes: float64(2), int64(2), object(1)
memory usage: 1.1+ KB


In [6]:
df.describe()

,time_point,steps_count,heart_rate,skin_temp
count,24.000000,24.000000,24.000000,24.000000
mean,11.500000,1340.000000,74.125000,36.850000
std,7.071068,1842.708309,18.101916,0.486335
min,0.000000,0.000000,55.000000,36.000000
25%,5.750000,0.000000,58.750000,36.475000
50%,11.500000,350.000000,70.000000,36.950000
75%,17.250000,2250.000000,85.000000,37.225000
max,23.000000,6000.000000,120.000000,37.600000


In [7]:
# 1. Workout heart rates

workout_hr = df.loc[df["event_flag"] == "workout", "heart_rate"]
print("Heart rate during workout:\n", workout_hr)

Heart rate during workout:
 7     120.0
18    110.0
Name: heart_rate, dtype: float64


In [11]:
# 1.1 Heart rate during workout hours (табличный вывод)
workout_hr_table = df.loc[df["event_flag"] == "workout", ["time_point", "heart_rate"]]

print("📌 Heart rate during workout:\n")
print(workout_hr_table.to_string(index=False))


📌 Heart rate during workout:

 time_point  heart_rate
          7       120.0
         18       110.0


In [14]:
# 1.2 Calculate the average skin_temp value during the hours when the event is 'normal'.
avg_temp_normal = df.loc[df["event_flag"] == "normal", "skin_temp"].mean()
print("🌡 Average skin temperature at 'normal':", round(avg_temp_normal, 2), "°C")


🌡 Средняя температура кожи при 'normal': 37.2 °C


In [16]:
# 2.1 Calculate the change in the number of steps per hour (the difference between the current and previous value)

df["step_change"] = df["steps_count"].diff()
print("📈 Changing the number of steps by hour:")
print(df[["time_point", "steps_count", "step_change"]].head(10))

📈 Изменение количества шагов по часам:
   time_point  steps_count  step_change
0           0            0          NaN
1           1            0          0.0
2           2            0          0.0
3           3            0          0.0
4           4            0          0.0
5           5            0          0.0
6           6           10         10.0
7           7          500        490.0
8           8         2000       1500.0
9           9         3000       1000.0


In [17]:
# 2.2 Find the hours when steps_count increased compared to the previous one.

increased_steps = df[df["step_change"] > 0]
print("🔼 Hours when the number of steps increased:")
print(increased_steps[["time_point", "steps_count", "step_change"]].to_string(index=False))

🔼 Часы, когда количество шагов увеличилось:
 time_point  steps_count  step_change
          6           10         10.0
          7          500        490.0
          8         2000       1500.0
          9         3000       1000.0
         10         4000       1000.0
         11         5000       1000.0
         12         6000       1000.0
         18          800        300.0


In [18]:
# 3.1 Step 1: Build a 2D array
hr_array = df[["time_point", "heart_rate"]].to_numpy()
print("📊 2D-array (time_point, heart_rate):")
print(hr_array[:5])  # покажем первые 5 строк


📊 2D-массив (time_point, heart_rate):
[[ 0. 60.]
 [ 1. 58.]
 [ 2. 59.]
 [ 3. 57.]
 [ 4. 56.]]


In [19]:
# 3.2 Step 2: Find your maximum heart rate hour
max_idx = df["heart_rate"].idxmax()
max_time = df.loc[max_idx, "time_point"]
max_hr = df.loc[max_idx, "heart_rate"]

print(f"💓 Maximum heart rate {max_hr} bpm был в {max_time}:00")

💓 Максимальный пульс 120.0 bpm был в 7:00


In [21]:
# 4.1 We believe that during sleep steps_count is incorrect. Replace all such values ​​with np.nan.

df["steps_cleaned"] = df["steps_count"].copy()
df.loc[df["event_flag"] == "sleep", "steps_cleaned"] = np.nan


In [22]:
# 4.2 Now recalculate the steps_count average, ignoring NaNs: np.nanmean(...).

avg_steps_cleaned = np.nanmean(df["steps_cleaned"])
print(f"🚶 Average number of steps (without sleep): {round(avg_steps_cleaned, 2)}")


🚶 Среднее количество шагов (без сна): 1891.76
